## How to generate integral matrices with neat reduced row echelon form / inverse if exists?

Assume we want to generate $A$, an integral $m\times n$ matrix of rank $k$. An approach could be:
1. Generate $R$, a reduced row echelon form of $A$ with all integral entries
2. Perform EROs on $R$ to form $A$
   - Type I: $r_i \leftrightarrow r_j$
   - Type II: $cr_i \rightarrow r_i$, where $c$ is a non-zero integer
   - Type III: $r_i+ar_j \rightarrow r_i$, where $a$ is an integer
 
To keep the problem neat and ensure sufficient level of difficulty, the following constraints are imposed in the following algorithm:
- Different ranges are set on the randomly generated values suggested
- Maximum number of zeros in Column 1 of the generated matrix $A$ is 1
- Maximum magnitude of the entries of $A$ is 20

Applications of the proposed algorithm:
- Generating any problems related to reduced row echelon form

In [1]:
# import necessary packages
import numpy as np
import sympy as sp
from sympy.printing.str import StrPrinter
import random
from numpy.random import randint
from scipy import stats

In [2]:
# Function that prints sympy matrices
def printM(M):
    printer = StrPrinter()
    print(M.table(printer, align='center'))

In [3]:
# Function that returns a rref according to the dimensions and rank input
def FormRREF(m, n, rank):
    
    M = sp.zeros(m,n)   # initialize the matrix M

    # draw the columns that contain a leading one
    leading = np.arange(0, n, 1)
    random.shuffle(leading)
    if (0 in leading[0:rank]):
        leading = leading[0:rank]
    else:
        leading[rank-1] = 0
        leading = leading[0:rank]

    # construct rref according to the leading array
    k = 0
    for j in range(n):
        if (j in leading):
            M[k,j] = 1
            k+=1
        else:
            for l in range(k):
                M[l,j] = randint(-5,5)
    
    return M

In [4]:
# Functions that defines EROs
def Type1(M):
    i = randint(0, M.shape[0])
    j = randint(0, M.shape[0])
    while (j == i):
        j = randint(0, M.shape[0])
    M.row_swap(i, j)

def Type2(M):
    i = randint(0, M.shape[0])
    c = randint(-8, 8)
    while (c == 0 or c == 1):    # 0 and 1 not allowed
        c = randint(-8, 8)
    M.row_op(i, lambda x, j: x*c)

def Type3(M):
    i = randint(0, M.shape[0])  # multiples of other rows to be added to row i
    arr = randint(-4, 4, M.shape[0])   # arr defines the how many times of other rows 
                                       # to be added (arr[i] will be ignored)
    while (not all(arr[i]!=0 for i in range(len(arr)))):
        arr = randint(-4, 4, M.shape[0])
    
    for j in range(M.shape[0]):
        if (i!=j):
            M.row_op(i, lambda x, y: x+arr[j]*M[j,y])

In [5]:
# Function that performs inverse EROs on a given matrix m x n matrix M
def InvERO(M, debug):
    
    # copy M to X
    X = sp.Matrix(M)
    
    m, n = X.shape
    
    # randomly assign the number of intended EROs
    # (not the actual one since a "Type3" ERO defined here can be written as
    # zero or more Type 3 EROs defined in class)
    steps = randint(m+n, 2*(m+n))
    
    # randomly draw the type of ERO in each step
    types = np.array([1,2,3])
    probs = (0.1, 0.2, 0.7)
    GenTypeList = stats.rv_discrete(values=(types, probs))   # Probability of drawing Type types[i] 
                                                             # = probs[i] for i = 1,2,3
    typeList = GenTypeList.rvs(size=steps)   # Draw the type of ERO for each step
    
    # perform EROs
    for k in range(steps):
        if (debug==True):
            printM(X)
            print("Step ", k+1, ": Type ", typeList[k])
            
        if (typeList[k]==1):
            Type1(X)
        elif (typeList[k]==2):
            Type2(X)
        else:
            Type3(X)
    
    if (debug==True):
        printM(X)
    
    # discard the generated matrix if there are too many zeros in Column 1
    count0 = 0
    for i in range(m):
        if (X[i,0]==0):
            count0 = count0 + 1
            if (count0 > 1):
                if (debug==True):
                    print("Too many zeros in Column 1, a new matrix will be generated")
                return InvERO(M,debug)
    
    # discard the generated matrix if some of the entries is strictly larger than 20 in magnitude
    for i in range(m):
            for j in range(n):
                if (abs(X[i,j]) > 20):
                    if (debug==True):
                        print("Resultant entries are too large, a new matrix will be generated")
                    return InvERO(M, debug)
    
    return X

In [6]:
# Main function
def MatrixGen(m, n, rank, debug = False):

    # check input
    if (rank > min(m,n)):
        print("Error: rank exceeds min{m,n}")
        return
    
    R = FormRREF(m, n, rank)  # form the reduced row echelon form R
    A = InvERO(R, debug)   # perform EROs to generate the problem matrix A
    
    return A

In [7]:
# For program testing
import time
tStart = time.time()

for i in range(5):  # parameters to be determined by user: number of problems to be generated
    
    # parameters to be determined by user: m, n, rank and debug
    m, n, rank, debug = 4, 4, 4, False
    
    X = MatrixGen(m, n, rank, debug)
    
    print("Problem")
    printM(X)
    
    if (m == n and n == rank):  # print the inverse of the generated matrix if the matrix is invertible
        print("Inverse")
        printM(X.inv())
    else:                       # print the rref of the generated matrix in other cases
        print("Reduced row echelon form")
        printM(X.rref()[0])
    print("")

tEnd = time.time()
print("Running time:", tEnd - tStart, "s")

Problem
[-1, 1 , 5 , -4]
[1 , 2 , 1 , 2 ]
[1 , -3, -1, 5 ]
[4 , 7 , -5, 9 ]
Inverse
[-138, 238, -67, -77]
[ 25 , -43,  12,  14]
[ 2  ,  -3,  1 ,  1 ]
[ 43 , -74,  21,  24]

Problem
[-12, 14, -7, 5 ]
[-11, 15, -6, 5 ]
[ 6 , -5, 4 , -8]
[ -3, 3 , -2, 7 ]
Inverse
[ 52/3, -27/2,  61/3,  41/2]
[  4  ,   -3 ,   5  ,   5  ]
[-67/3,  35/2, -76/3, -51/2]
[ -2/3,  1/2 ,  -2/3,  -1/2]

Problem
[0 ,  -4, -1,  7 ]
[2 ,  7 , 1 , -11]
[1 ,  5 , 1 ,  -8]
[-4, -12, -2,  19]
Inverse
[0, -1, -1, -1]
[3, 1 , 6 , 2 ]
[1, -4, 4 , -1]
[2, 0 , 4 , 1 ]

Problem
[-1, -1,  10, -1]
[1 , -2,  6 , -1]
[0 , 0 ,  -2, 0 ]
[2 , 1 , -16, 1 ]
Inverse
[1 , 0 ,  -3 , 1 ]
[3 , -1,  -4 , 2 ]
[0 , 0 , -1/2, 0 ]
[-5, 1 ,  2  , -3]

Problem
[-2, 2 , 1 , 1 ]
[18, 0 , 0 , 0 ]
[5 , 11, -2, -1]
[4 , 9 , 7 , 7 ]
Inverse
[ 0  ,  1/18 , 0 ,   0  ]
[7/5 ,  1/5  , 0 ,  -1/5]
[86/5, 83/30 , -1, -13/5]
[-19 , -55/18, 1 ,   3  ]

Running time: 2.226231098175049 s
